In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('all_data.csv')

In [4]:
data.head()

,Unnamed: 0,Дата рождения,Возраст,Пол,ECOG (до лечения),Дата биопсии (при пункции/при лапароскопии),Основной диагноз,Сопутствующая патология,T,N,...,ECOG (до АДХТ),Вес (до АХТ),Потеря веса (от момента НАХТ до АДХТ),ИМТ (до АДХТ),Общ белок (до АДХТ),АЛТ (до АДХТ),АСТ (до АДХТ),ЛДГ (до АДХТ),Диагноз (обновленный,Гистология (обновленная)
0,0,07.08.55 00:00,59,М,1,12.02.15 00:00,ПММР. Рак головки поджелудочной железы. Рак мо...,Менингиома головного мозга.,T3,Nx,...,1,82,0,"25,9",66,100,46,559,Рак головки,NaN
1,1,01.01.45 00:00,70,Ж,2,14.09.15 00:00,Рак головки поджелудочной железы T3N1M0. Состо...,ИБС. Атеросклеротический кардиосклероз. Гиперт...,T3,N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Рак головки,NaN
2,2,30.12.99 00:00,61,М,2,08.09.15 00:00,Рак тела поджелудочной железы T4NXM0 Стадия: н...,Инсульт геморагический. Ущемленная пупочная гр...,T4,Nx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Рак тела,NaN
3,3,29.03.55 00:00,60,Ж,1,22.09.15 00:00,рак тела и хвоста поджелудочной железы,нет,T3,Nx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Сочетанное поражение,NaN
4,4,16.04.47 00:00,68,Ж,1,09.10.15 00:00,Рак головки поджелудочной железы сT4N1M0,Гипертоническая болезнь сердца,T4,N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Рак головки,NaN


Исследуем данные.

In [31]:
print "пациентов: {0}".format(data.shape[0])
print "признаков: {0}".format(data.shape[1])

пациентов: 129
признаков: 108


Это тот набор фичей, который выделил врач, который предоставил данные. <br\>
Если я все правильно понимаю, нужно научиться выбирать правильную первую линию ХТ ("Схема лечения (1 линия ХТ)") <br>
Характеристикой качества работы лечения видимо выступает время жизни пациентов (от наблюдения до момента закрытия карты), эти данные не обновлены. <br>
Перевая линия ХТ прописана у 24 пациентов из 129, у остальных пропуски. <br>


In [32]:
main_features = [
    "Код",
    "Возраст",
    "Пол",
    "Дата установления диагноза",
    "Сопутствующая патология",
    "Стадия",
    "Желчеразгрузочная операция",
    "Дата желчеразгрузочной операции",
    "Гистология опухоли (до лечения)",
    "ECOG (до лечения)",
    "Вес (до лечения)",
    "ИМТ (до лечения)",
    "Общ белок (до лечения)",
    "АЛТ (до лечения)",
    "АСТ (до лечения)",
    "ЛДГ (до лечения)",
    "CA 19-9 (до лечения)",
    "РЭA (до лечения)",
    "Схема лечения (1 линия ХТ)",
    "Дата начала (1 линия ХТ)",
    "Диагноз (обновленный"
]

In [23]:
main_data = data[main_features]

Вот так выглядит статистика по первой линии ХТ, т.е. два основных препарата, пропусков 82%

In [39]:
main_data['Схема лечения (1 линия ХТ)'].value_counts()

Gemcitabin                                                                                                                                                     8
FOLFIRINOX                                                                                                                                                     8
FOLFOX                                                                                                                                                         2
гемзар                                                                                                                                                         1
Gemcitabin + cisplatin                                                                                                                                         1
Gemcitabin + паклитаксел                                                                                                                                       1
AF                                

Пропусков в остальных данных

In [86]:
for col in main_data.columns:
    print "{0}: {1} {2}%".format(col, (30 - (len(col) +  col.count(' '))/2)*" ", 1 - float(main_data[col].count()) / main_data[col].size)

Код:                             0.0%
Возраст:                         0.046511627907%
Пол:                             0.0%
Дата установления диагноза:      0.062015503876%
Сопутствующая патология:         0.232558139535%
Стадия:                          0.0697674418605%
Желчеразгрузочная операция:      0.62015503876%
Дата желчеразгрузочной операции:  0.651162790698%
Гистология опухоли (до лечения):  0.348837209302%
ECOG (до лечения):                  0.093023255814%
Вес (до лечения):                 0.077519379845%
ИМТ (до лечения):                 0.093023255814%
Общ белок (до лечения):           0.077519379845%
АЛТ (до лечения):                 0.093023255814%
АСТ (до лечения):                 0.093023255814%
ЛДГ (до лечения):                 0.961240310078%
CA 19-9 (до лечения):                0.759689922481%
РЭA (до лечения):                  0.790697674419%
Схема лечения (1 линия ХТ):        0.813953488372%
Дата начала (1 линия ХТ):          0.813953488372%
Диагноз (обновленный:

In [122]:
anti_mask = np.array(main_data['Схема лечения (1 линия ХТ)'].isnull())
mask = np.logical_not(anti_mask)

Вот так выглядят пропуски, в той части данных где целевая переменная не пропуск

In [129]:
for col in main_data[mask].columns:
     print "{0}: {1} {2}%".format(col, (30 - (len(col) +  col.count(' '))/2)*" ", 1 - float(main_data[mask][col].count()) / main_data[mask][col].size)

Код:                             0.0%
Возраст:                         0.0%
Пол:                             0.0%
Дата установления диагноза:      0.0%
Сопутствующая патология:         0.25%
Стадия:                          0.0%
Желчеразгрузочная операция:      0.5%
Дата желчеразгрузочной операции:  0.583333333333%
Гистология опухоли (до лечения):  0.166666666667%
ECOG (до лечения):                  0.0%
Вес (до лечения):                 0.0%
ИМТ (до лечения):                 0.0%
Общ белок (до лечения):           0.0%
АЛТ (до лечения):                 0.0833333333333%
АСТ (до лечения):                 0.0833333333333%
ЛДГ (до лечения):                 0.833333333333%
CA 19-9 (до лечения):                0.25%
РЭA (до лечения):                  0.375%
Схема лечения (1 линия ХТ):        0.0%
Дата начала (1 линия ХТ):          0.0416666666667%
Диагноз (обновленный:             0.166666666667%
